In [147]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [148]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-19 10:29:06--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   158MB/s    in 0.2s    

2024-01-19 10:29:06 (158 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [149]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [150]:
# add your code here - consider creating a new cell for each section of code
userCounts = df_ratings['user'].value_counts()
isbnCounts = df_ratings['isbn'].value_counts()
# remove all users with less than 200 reviews
df_ratings = df_ratings[~df_ratings['user'].isin(userCounts[userCounts < 200].index)]
# remove all books with less than 100 ratings
df_ratings = df_ratings[~df_ratings['isbn'].isin(isbnCounts[isbnCounts < 100].index)]

In [151]:
df_combined = df_ratings.merge(df_books, left_on = 'isbn', right_on = 'isbn', how = 'left')

r = df_combined[['title', 'user']]
r = r.duplicated()
df_combined = df_combined[~r==True]
df_combined = df_combined[pd.notnull(df_combined['title'])]

df_combined_pivot = df_combined.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_combined_matrix = csr_matrix(df_combined_pivot.values)

In [152]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(df_combined_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [153]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = model_knn.kneighbors(df_combined_pivot.iloc[list(df_combined_pivot.index.values).index(book),:].values.reshape(-1,888), n_neighbors = 6)
  distances = distances[0][1:]
  titles = list(df_combined_pivot.iloc[indices.flatten()[1:],:].index)
  recommended_books = [df_combined_pivot.iloc[indices.flatten(),:].index[0], [[titles[i], distances[i]] for i in range(4,1,-1)]]
  return recommended_books

In [154]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411]]]
You passed the challenge! 🎉🎉🎉🎉🎉
